In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the data
data = pd.read_csv('continuous dataset.csv')

# Convert timestamp to datetime format
data['datetime'] = data['datetime'][:45261].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))

# Set the datetime as index
data.set_index('datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['nat_demand'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['nat_demand'].rolling(window=24).mean()
data['rolling_std'] = data['nat_demand'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]/1000
test_data = data[train_size:]/1000


In [2]:
#SARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Fit the SARIMA model
model = SARIMAX(train_data['nat_demand'], order=(5, 1, 0), seasonal_order=(0, 1, 1, 24))
model_fit = model.fit()

# Make predictions on the test set
predictions = model_fit.predict(start=len(train_data), end=len(data)-1)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(test_data['nat_demand'], predictions)
mse = mean_squared_error(test_data['nat_demand'], predictions)
rmse = np.sqrt(mse)

print('MAE:', mae)
print('MSE:', mse)
print('RMSE:', rmse)
ypred=predictions
y_test=test_data['nat_demand']

#For MAE
mae=np.mean(np.abs(ypred-y_test))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
def mean_absolute_percentage_error(y_test, ypred): 
    return np.mean(np.abs((y_test - ypred)/y_test))*100.
mape = mean_absolute_percentage_error(y_test, ypred)
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))


C:\Users\gulsh\anaconda32\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
C:\Users\gulsh\anaconda32\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


MAE: 0.6874834605725029
MSE: 0.6287148061691714
RMSE: 0.7929153839907329
MAE = 0.69 (%)
MAPE = 56.81 (%)
RSME = 0.79 (%)
